In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [58]:
import numpy as np
import pandas as pd

df = pd.read_csv('C:/ds_work/source_code/Project_DS/data/train_NullAll_update.csv')
df_train = df
df_train

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,1.0,0.0,1.0,0.0,0.0,2.0,39.0,0.0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,0.0
1,0002_01,0.0,0.0,5.0,0.0,1.0,2.0,24.0,0.0,109.0,9.0,25.0,549.0,44.0,Juanna Vines,1.0
2,0003_01,1.0,0.0,0.0,0.0,1.0,2.0,58.0,1.0,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,0.0
3,0003_02,1.0,0.0,0.0,0.0,1.0,2.0,33.0,0.0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,0.0
4,0004_01,0.0,0.0,5.0,1.0,1.0,2.0,16.0,0.0,303.0,70.0,151.0,565.0,2.0,Willy Santantines,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,8772_02,1.0,0.0,3.0,90.0,0.0,0.0,53.0,0.0,0.0,1127.0,0.0,3939.0,400.0,Naosura Motled,0.0
8689,3821_01,0.0,0.0,4.0,309.0,0.0,1.0,35.0,0.0,0.0,2.0,0.0,0.0,867.0,Violan Mcphernard,0.0
8690,7746_01,1.0,1.0,4.0,289.0,0.0,0.0,35.0,0.0,0.0,0.0,0.0,0.0,0.0,Antinon Patoetic,1.0
8691,4167_01,0.0,0.0,4.0,309.0,0.0,1.0,33.0,0.0,0.0,440.0,0.0,0.0,334.0,Ninaha Deckerson,0.0


In [59]:
df_train.isnull().sum()

PassengerId       0
HomePlanet        0
CryoSleep         0
Cabin             0
Cabin_num         0
Cabin_port        0
Destination       0
Age               0
VIP               0
RoomService       0
FoodCourt         0
ShoppingMall      0
Spa               0
VRDeck            0
Name            200
Transported       0
dtype: int64

In [54]:
df_train['Cabin_port'].max()

1.0

In [55]:
# train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import xgboost as xgb
from xgboost import XGBRegressor


def fillnull(column_name):

    global X_train, X_test, y_train, y_test

    # df_FoodCourt =  df_train중에서 FoodCourt null값이 존재하는 열만 추출 
    # -> 훈련된 모델의 값으로 다시 채울 예정
    df_column_name = df_train[df_train[column_name].isnull()]
    #df_train 중에서 FoodCourt 의 null값이 존재하는 열만 추출 -> 추후 훈련할 데이터프레임
    df_train01 = df_train.dropna(subset=[column_name])
    # df_train
    df_train[df_train[column_name].isnull()]

    X_train, X_test, y_train, y_test = train_test_split(df_train01.drop(['PassengerId',column_name,'Name'],axis=1),df_train01[column_name], test_size=0.2, random_state=42)

    # scaler
    scaler = StandardScaler().fit(X_train)
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # 모델
    model = xgb.XGBRegressor(early_stopping_rounds=10)
    eval_set = [(X_test, y_test)]
    model.fit(X_train, y_train, eval_set=eval_set, verbose=False)#, verbose=True

    pred = model.predict(X_test)

    global df_update
    # df_01 은 df_train 중에서 FoodCourt 의 값이 null인 경우의 데이터프레임
    df_column_name_null = df_column_name.drop(column_name,axis=1)
    # 훈련된 xgboost모델을 사용해서 df_homeplanet_null의 HomePlanet값을 예측
    pred = model.predict(df_column_name_null.drop(['PassengerId','Name'],axis=1))
    pred.max()
    pred = np.round(pred)

    # 예측된 값을 다시 df_homeplanet으로 삽입(null값 채우기)
    df_column_name[column_name] = pred
    df_column_name
    
    # null값을 채운 dataframe과 df_train01 합치기 
    df_update = pd.concat([df_train01,df_column_name],axis=0)

fillnull('Cabin_port')


C:\Users\Public\Documents\ESTsoft\CreatorTemp\ipykernel_26488\1866386500.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_column_name[column_name] = pred


In [56]:
df_update[df_update['Cabin_port'].isnull()]

,PassengerId,HomePlanet,CryoSleep,Cabin,Cabin_num,Cabin_port,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported


In [57]:
df_update.to_csv('C:/ds_work/source_code/Project_DS/data/train_NullAll_update.csv', index=False)
